In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

import re

def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []
    
def parse_edge(s):
  user, follower = s.split("\t")
  return (int(user), int(follower))

#just reading the file

edges = sc.textFile("/data/twitter/twitter_sample_small.txt").map(parse_edge).cache()

In [3]:
# simple function to split list into ints and take 1st

def separate_num(x):
    return x[0]

# defining additional variables
d = 1
res_str = ''

# first step: we don't need entire datatset as for now, only those pairs where 12 is a follower
e_12 = edges.filter(lambda e: e[1] == 12).cache()

# simple check for simple tests, maybe 12 is already following 34?
is_34 = e_12.filter(lambda e: e[0] == 34).cache().count()
if is_34 > 0:
    res_str = res_str + '12,34' 
    print res_str;
#if no then here we go:
else:
    # taking those who are followed by 12
    e_12_first = e_12.map(separate_num)
    
    # convert them into integers
    e_12_c = e_12_first.collect()
    
    # initiating RDD for loop
    prev_in_loop = e_12.persist()
    
    # filtering all pairs where edge is followed by edge followed by 12:
    temp_in_loop = edges.filter(lambda e: e[1] in e_12_c).persist()
    
    # saving the join result into RDD, exchanging always in the way to place the next edge as 1st in pair
    join = edges.filter(lambda e: e[1] in e_12_c).map(lambda e: (e[1], e[0])).join(prev_in_loop) \
        .map(lambda (x,y) : (y[0], (x, y[1]))).persist()
    
    # starting loop
    while (True):
        # Spark is a "lazy- computing" so it works based on links, by trigerring code below I do re-compute dataset
        # based on previous result
        e_12_first = temp_in_loop.map(separate_num).persist()
        e_12_c = e_12_first.collect()
        # Refreshing temp valiables and extending join loop
        
        temp_in_loop = edges.filter(lambda e: e[1] in e_12_c).persist()
        join = temp_in_loop.map(lambda e: (e[1], e[0])).join(join) \
        .map(lambda (x,y) : (y[0], (x, y[1]))).persist()
        
        # Checking do we point to the end edge?
        is_34 = temp_in_loop.filter(lambda e: e[0] == 34).cache().count()
        if is_34 > 0:
            result = join.filter(lambda e: e[0] == 34).take(1)
            break;
        # If no, keep loop until we do
        d = d+1
        prev_in_loop = temp_in_loop.persist()

# Code above might loop a bit tricky, you need to run it and see step by step how it goes
# Now time to parse the result of nested tuples into single string
        
    res_str = res_str + str(result[0][0])
    result = result[0][1]
    while True:
        try:
            Ptarget = int(result[1])
            res_str = res_str + ',' + str(result[0]) + ',' + str(result[1])
            break;
        except:
            res_str = res_str + ',' + str(result[0])
            result = result[1]

In [4]:
# And since recieved before string is swapped, we do swap it back

vertex = res_str.split(",")

i = len(vertex)
j = 2
final_str = '12'
while (j < i + 1):
    final_str = final_str + ',' +  vertex[i-j]
    j = j + 1
    
# Result!

print final_str

12,422,53,52,107,20,23,274,34
